## CFM 301 DA4
### Q6
#### Jeongseop Yi (Patrick), j22yi

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm

In [3]:
q6_df = pd.read_sas('q6_data.sas7bdat')
q6_df['date'] = pd.to_datetime(q6_df['DATE'])
q6_df['year'] = q6_df['date'].dt.year
q6_df['month'] = q6_df['date'].dt.month
q6_df['yyyymm'] = q6_df['year'].astype(str) + q6_df['month'].astype(str).str.zfill(2)

q6_df.columns = list(map(str.lower, q6_df.columns))
q6_df = q6_df.drop_duplicates(subset=['permno', 'yyyymm'])
q6_df['prc'] = q6_df['prc'].abs()
q6_df['mktcap'] = q6_df['prc'] * q6_df['shrout']
q6_df['lnSize'] = np.log(q6_df['mktcap'])
q6_df['prc_prev'] = q6_df.groupby(['permno'])['prc'].shift(1)
q6_df['mktcap_prev'] = q6_df.groupby(['permno'])['mktcap'].shift(1)
q6_df['ret_t1'] = q6_df.groupby(['permno'])['ret'].shift(-1)

q6_df = q6_df[(q6_df['mktcap_prev'] >= 100 * 1000) & (q6_df['prc_prev'] >= 5)]

In [4]:
# winsorize log market cap by 3 standard deviations
q6_df['lnSize_w'] = q6_df.groupby('yyyymm')['lnSize'].transform(lambda x: x.clip(x.mean() - 3 * x.std(), x.mean() + 3 * x.std()))

In [5]:
def get_monthly_returns_quintile(df, col):
    df[col + '_q'] = df.groupby('yyyymm')[col].transform(lambda x: pd.qcut(x, 5, labels=False))
    df[col + '_q_ret'] = df.groupby(['yyyymm', col + '_q'])['ret_t1'].transform('mean')
    # create data frame with monthly returns for each quintile
    df_q_ret = pd.DataFrame(columns=['1', '2', '3', '4', '5'])
    for i in range(1, 6):
        df_q_ret[str(i)] = df[df[col + '_q'] == i - 1].groupby('yyyymm')[col + '_q_ret'].mean()
    df_q_ret = df_q_ret.reset_index()
    return df, df_q_ret

In [6]:
factors = ['lnSize_w']
factors_quinlst = []
q6_df.set_index('yyyymm', inplace=True)
for factor in factors:
    q6_df, factor_ret = get_monthly_returns_quintile(q6_df, factor)
    factors_quinlst.append(factor_ret)
q6_df.reset_index(inplace=True)

In [7]:
factors_port = pd.DataFrame(columns=['yyyymm'].append(factors))
factors_port['yyyymm'] = factors_quinlst[0]['yyyymm']
for i in range(1, 6):
    factors_port['q' + str(i)] = factors_quinlst[0][str(i)]

In [8]:
display(factors_port)

,yyyymm,q1,q2,q3,q4,q5
0,199902,-0.032268,-0.008022,0.013687,0.035808,0.034982
1,199903,0.096820,0.096985,0.077759,0.083797,0.080085
2,199904,0.031442,0.032844,0.027911,0.010931,-0.012922
3,199905,0.036847,0.074642,0.074998,0.063486,0.040523
4,199906,0.017213,-0.015422,-0.019305,-0.020435,-0.033509
...,...,...,...,...,...,...
270,202108,-0.013888,-0.026068,-0.027152,-0.031019,-0.038635
271,202109,0.008797,0.023755,0.023472,0.038369,0.059465
272,202110,-0.042329,-0.044006,-0.056072,-0.048098,-0.037505
273,202111,-0.014024,-0.011191,0.003711,0.023247,0.038584


In [9]:
ff4_df = pd.read_sas('ff4.sas7bdat')
ff4_df.columns = map(str.lower, ff4_df.columns)
ff4_df['date'] = pd.to_datetime(ff4_df['dateff'])
ff4_df['year'] = ff4_df['date'].dt.year
ff4_df['month'] = ff4_df['date'].dt.month
ff4_df['yyyymm'] = ff4_df['year'].astype(str) + ff4_df['month'].astype(str).str.zfill(2)
ff4_df = ff4_df[(ff4_df['yyyymm'] >= '200001') & (ff4_df['yyyymm'] <= '202111')]

factors_port = pd.merge(factors_port, ff4_df, on='yyyymm', how='left')
factors_port = factors_port[(factors_port['yyyymm'] >= '200001') & (factors_port['yyyymm'] <= '202111')]

In [10]:
display(factors_port)

,yyyymm,q1,q2,q3,q4,q5,dateff,smb,hml,mktrf,rf,umd,date,year,month
11,200001,0.083245,0.132900,0.119409,0.098660,0.059526,2000-01-31,0.0579,-0.0191,-0.0474,0.0041,0.0192,2000-01-31,2000.0,1.0
12,200002,0.008408,-0.006863,0.005621,0.022983,0.024381,2000-02-29,0.2142,-0.0970,0.0245,0.0043,0.1820,2000-02-29,2000.0,2.0
13,200003,-0.121977,-0.106721,-0.097140,-0.106645,-0.055149,2000-03-31,-0.1723,0.0817,0.0520,0.0047,-0.0683,2000-03-31,2000.0,3.0
14,200004,-0.113128,-0.077706,-0.054578,-0.047731,-0.030319,2000-04-28,-0.0668,0.0726,-0.0640,0.0046,-0.0839,2000-04-28,2000.0,4.0
15,200005,0.131408,0.179873,0.126250,0.085774,0.038808,2000-05-31,-0.0609,0.0481,-0.0442,0.0050,-0.0898,2000-05-31,2000.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,202107,0.024316,0.029813,0.016675,0.017584,0.023871,2021-07-30,-0.0399,-0.0178,0.0127,0.0000,-0.0225,2021-07-30,2021.0,7.0
270,202108,-0.013888,-0.026068,-0.027152,-0.031019,-0.038635,2021-08-31,-0.0043,-0.0015,0.0291,0.0000,0.0251,2021-08-31,2021.0,8.0
271,202109,0.008797,0.023755,0.023472,0.038369,0.059465,2021-09-30,0.0072,0.0509,-0.0437,0.0000,0.0149,2021-09-30,2021.0,9.0
272,202110,-0.042329,-0.044006,-0.056072,-0.048098,-0.037505,2021-10-29,-0.0234,-0.0049,0.0665,0.0000,0.0319,2021-10-29,2021.0,10.0


In [11]:
summary_df = pd.DataFrame(columns=['index'].append(factors))
summary_df['index'] = ['overall return', 'yearly overall return', 'excess return',
                       'CAPM alpha', 'CAPM t-stat', 'FF4 alpha', 'FF4 t-stat', 'Sharpe']
summary_df.set_index('index', inplace=True)
for i in range(1, 6):
    mkt = sm.add_constant(factors_port['mktrf'])
    model = sm.OLS(factors_port['q' + str(i)] - factors_port['rf'], mkt).fit()
    model_alpha = model.params[0]
    model_tstat = model.tvalues[0]

    ff4 = sm.add_constant(factors_port[['mktrf', 'smb', 'hml', 'umd']])
    model_ff4 = sm.OLS(factors_port['q' + str(i)] - factors_port['rf'], ff4).fit()
    model_ff4_alpha = model_ff4.params[0]
    model_ff4_tstat = model_ff4.tvalues[0]

    summary_df['q' + str(i)] = [factors_port['q' + str(i)].mean(),
                                factors_port['q' + str(i)].mean() * 12,
                                (factors_port['q' + str(i)] - factors_port['rf'] - (factors_port['mktrf'])).mean(),
                                model_alpha,
                                model_tstat.round(2),
                                model_ff4_alpha,
                                model_ff4_tstat.round(2),
                                (factors_port['q' + str(i)] - factors_port['rf']).mean() / (factors_port['q' + str(i)] - factors_port['rf']).std() * np.sqrt(12)]

display(summary_df.round(4).T)

index,overall return,yearly overall return,excess return,CAPM alpha,CAPM t-stat,FF4 alpha,FF4 t-stat,Sharpe
q1,0.0094,0.1125,0.0022,0.0069,1.79,0.0073,1.86,0.4481
q2,0.0090,0.1079,0.0018,0.0068,1.67,0.0073,1.77,0.4095
q3,0.0089,0.1068,0.0017,0.0069,1.77,0.0074,1.87,0.4211
q4,0.0087,0.1041,0.0015,0.0068,1.96,0.0073,2.06,0.4575
q5,0.0082,0.0984,0.0010,0.0064,2.12,0.0068,2.23,0.4965


In [14]:
q6_df_summary = factors_port[['q1', 'q2', 'q3', 'q4', 'q5']].describe(percentiles=[0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99])
display(q6_df_summary.reset_index())
with pd.ExcelWriter("../DA4_data.xlsx", mode='a', engine="openpyxl", if_sheet_exists='replace') as writer:
    q6_df_summary.reset_index().to_excel(writer, sheet_name="q6", index=False)

,index,q1,q2,q3,q4,q5
0,count,263.000000,263.000000,263.000000,263.000000,263.000000
1,mean,0.009374,0.008988,0.008899,0.008671,0.008204
2,std,0.062673,0.065342,0.062776,0.056045,0.048360
3,min,-0.225674,-0.253409,-0.236774,-0.226912,-0.211207
4,1%,-0.152933,-0.155183,-0.152811,-0.124719,-0.124058
5,5%,-0.099623,-0.104616,-0.096734,-0.086915,-0.080508
6,25%,-0.024174,-0.031090,-0.027697,-0.022557,-0.016270
7,50%,0.011042,0.012033,0.014317,0.015149,0.012301
8,75%,0.047711,0.054362,0.046728,0.042751,0.038580
9,95%,0.095912,0.099615,0.096283,0.084829,0.075018
